In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="dangvantuan/sentence-camembert-large")

In [ ]:
from qdrant_client import QdrantClient

qdrant = QdrantClient(path="../qdrant_data") #Switched from using qdrant 'production mode' to 'embedded mode'

In [ ]:
import json
import uuid
import hashlib
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

# Helper function to convert string IDs to valid UUID-compatible IDs
def string_to_uuid(string_id):
    """Convert a string to a deterministic UUID by hashing it"""
    # Create MD5 hash of the string
    hash_object = hashlib.md5(string_id.encode())
    # Convert to hex
    hex_dig = hash_object.hexdigest()
    # Create a UUID from the hex string
    return uuid.UUID(hex_dig)

In [ ]:
try:
    with open('output_chunks.json', 'r', encoding='utf-8') as f:
        documents = json.load(f)
    print(f"✓ Loaded {len(documents)} documents from output_chunks.json")
except FileNotFoundError:
    print("Error: output_chunks.json file not found!")
    sys.exit(1)
except json.JSONDecodeError:
    print("Error: Invalid JSON format in output_chunks.json!")
    sys.exit(1)

# 5. Create a collection
collection_name = "Auditron_legal_chunks"
print(f"Creating collection '{collection_name}'...")
qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=model.get_sentence_embedding_dimension(), distance=Distance.COSINE),
)

# 6. Process the documents
print("Processing documents...")
total_chunks = sum(len(doc.get("chunks", [])) for doc in documents)
batch_size = 50
processed_count = 0
error_count = 0
batch_points = []
id_mapping = {}  # To store mapping between original IDs and UUID IDs

for doc_idx, doc in enumerate(documents):
    print(f"Processing document {doc_idx+1}/{len(documents)}")
    chunks = doc.get("chunks", [])
    
    for chunk_idx, chunk in enumerate(chunks):
        try:
            # Extract text and ID
            text = chunk.get("text", "")
            original_id = chunk.get("chunk_id", f"unknown_{doc_idx}_{chunk_idx}")
            
            if not text.strip():  # Skip empty chunks
                print(f"Warning: Empty text in chunk {original_id}")
                continue
            
            # Clean and truncate text if necessary to avoid index errors
            # Some models have maximum input length limitations
            max_text_length = 512  # Adjust based on your model's limitations
            text = text.strip()[:max_text_length]
            
            # Convert string ID to UUID
            point_id = string_to_uuid(original_id)
            
            # Save mapping
            id_mapping[str(point_id)] = original_id
            
            # Generate embedding with error handling
            try:
                embedding = model.encode(text, show_progress_bar=False)
            except Exception as embed_error:
                print(f"Embedding error for chunk {original_id}: {str(embed_error)}")
                # Try with a shorter text if it might be a length issue
                if len(text) > 200:
                    try:
                        shorter_text = text[:200]
                        print(f"Retrying with shorter text for {original_id}")
                        embedding = model.encode(shorter_text, show_progress_bar=False)
                    except Exception as retry_error:
                        print(f"Still failed with shorter text: {str(retry_error)}")
                        error_count += 1
                        continue
                else:
                    error_count += 1
                    continue
            
            # Create point with UUID
            point = PointStruct(
                id=str(point_id),
                vector=embedding.tolist(),
                payload={
                    "text": text,
                    "original_id": original_id,  # Keep original ID in payload
                    "structures": chunk.get("structures", []),
                    "document_path": chunk.get("document_path", []),
                    "metadata": chunk.get("metadata", {})
                }
            )
            
            # Add to batch
            batch_points.append(point)
            processed_count += 1
            
            # If batch is full, upload to Qdrant
            if len(batch_points) >= batch_size:
                qdrant.upsert(
                    collection_name=collection_name,
                    points=batch_points,
                )
                print(f"Uploaded batch: {processed_count}/{total_chunks} chunks ({error_count} errors so far)")
                batch_points = []
                
        except Exception as e:
            print(f"Error processing chunk {original_id}: {str(e)}")
            error_count += 1

# Upload any remaining points
if batch_points:
    qdrant.upsert(
        collection_name=collection_name,
        points=batch_points,
    )
    print(f"Uploaded final batch: {processed_count}/{total_chunks} chunks")

# Save ID mapping for reference (optional)
try:
    with open('id_mapping.json', 'w', encoding='utf-8') as f:
        json.dump(id_mapping, f, indent=2)
    print("✓ Saved ID mapping to id_mapping.json")
except Exception as e:
    print(f"Warning: Could not save ID mapping: {str(e)}")

print(f"✅ Successfully processed {processed_count}/{total_chunks} chunks into Qdrant collection '{collection_name}'!")
print(f"Total errors encountered: {error_count}")